# Intialisation Steps
Connecting to Snowflake

## Importing required modules

In [93]:
import pandas as pd
from drg_connect import Snowflake
import qgrid 
from datetime import timedelta, datetime
import math

import warnings
warnings.filterwarnings('ignore')

## Snowflake connection parameter

In [94]:
##defining parameters of snowflake
snow = Snowflake(role = 'RWD_ANALYTICS_RW',database='SANDBOX_ANALYTICS',schema = 'SANDBOX')
engine = snow.engine

%reload_ext sql_magic
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine

# Queries from here

# Pelican EHR data

## Pelican loinc table details

In [39]:
%%read_sql

select * from RWD_DB.RWD.PELICAN_E_LOINC limit 3

Query started at 10:43:18 AM India Standard TimeInitiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
; Query executed in 0.20 m

,loinc_num,component,property,time_aspct,system,scale_typ,method_typ,class,last_modified,chng_type,...,document_section,example_ucum_units,example_si_ucum_units,status_reason,status_text,change_reason_public,common_test_rank,common_order_rank,common_si_test_rank,hl7_attachment_structure
0,9256-9,Fluid output.total,Vol,Pt,^Patient,Qn,Estimated,IO_OUT.ATOM,1996-09-06,ADD,...,None,None,None,None,None,None,0,0,0,None
1,38474-3,Acylcarnitine,SCnc,Pt,Bld.dot,Qn,None,CHEM,2008-10-21,NAM,...,None,umol/L,None,None,None,None,0,0,0,None
2,2404-2,Hemopexin,MCnc,Pt,Urine,Qn,None,CHEM,2006-10-26,MAJ,...,None,mg/dL,None,None,None,None,0,0,0,None


## Pelican laborder table

In [4]:
%%read_sql

select * from RWD_DB.RWD.PELICAN_LABORDER limit 3

Query started at 04:54:02 PM India Standard Time; Query executed in 0.14 m

,laborder_id,patient_id,provider_id,vendor_id,loinc_num,result_status,report_date,observed_at,obs_quan,obs_qual,unit,is_abnormal,abnormal_flag,created_at,last_modified,transcript_id
0,1364935398341433546,393E2EED-4D2C-79C7-D150-08E8154D142F,None,8844736773880785995,2160-0,Final,2015-11-08,2015-11-14,0.96,NaN,mg/dL,None,None,2015-11-14,2015-11-14,None
1,4177387127494394539,393E2EED-4D2C-79C7-D150-08E8154D142F,D8E5640E-9F96-E7A1-BA57-6E996DF43374,8844736773880785995,43396-1,Final,2018-07-31,2018-08-04,212.00,NaN,mg/dL (calc),None,None,2018-08-05,2018-08-05,None
2,-8421755992989376358,393E2EED-4D2C-79C7-D150-08E8154D142F,D8E5640E-9F96-E7A1-BA57-6E996DF43374,8844736773880785995,30446-9,Not available,2018-07-31,2018-08-04,NaN,281266006.0,cells/uL,None,None,2018-08-05,2018-08-05,None


## Corresponding loinc codes for triglycerides

### Things to remember about loinc codes
 - Format: nnnnn-n
 - Component: what was measured
 - Unit: check from laborder table
 - date: ask/confirm which date to use - report date or observed_at date
 - there's no 'component' variable in laborder table so create an excel for loinc codes and upload

In [15]:
%%read_sql

create or replace table st_ref_loinc as

    select
        loinc_num,
        component

    from RWD_DB.RWD.PELICAN_E_LOINC
    
    where component ilike '%triglyceride%'
        or component ilike '%hdl%'
        or component ilike '%cholesterol%'
        and not component ilike '%ldl%'

Query started at 08:03:17 PM India Standard Time; Query executed in 0.13 m

,status
0,Table ST_REF_LOINC successfully created.


In [16]:
snow.select("select * from st_ref_loinc")

,loinc_num,component
0,2086-7,Cholesterol.in HDL
1,50223-7,Cholesterol crystals
2,2087-5,Cholesterol.in IDL
3,57937-5,Cholesterol.in HDL 3a
4,32289-1,7-Dehydrocholesterol
5,17081-1,Triglyceride^post CFst
6,34472-1,Cholesterol crystals
7,35200-5,Cholesterol
8,2565-0,Cholesterol
9,14438-6,Cholesterol


## Reference table for triglyceride-related loinc codes

In [15]:
%%read_sql

create or replace temporary table st_loinc_ref1 as

select
    'loinc' as cat1,
    'tg' as cat2,
    null as cat3,
     loinc_num as value,
    RELATEDNAMES2 as description,
    current_date() as active_date,
    'swar' as active_reason,
    null as deactive_date,
    null as deactive_reason,
    'RWD_DB.RWD.PELICAN_E_LOINC' as source
from 
   RWD_DB.RWD.PELICAN_E_LOINC
    where
        component ilike '%triglyceride%'

Query started at 07:50:07 PM India Standard Time; Query executed in 0.12 m

,status
0,Table ST_LOINC_REF1 successfully created.


In [16]:
%%read_sql df

select * from st_loinc_ref1

Query started at 07:50:17 PM India Standard Time; Query executed in 0.07 m

,cat1,cat2,cat3,value,description,active_date,active_reason,deactive_date,deactive_reason,source
0,loinc,tg,None,17081-1,Trigl; TG; Trigly; Triglycrides; Trig; Triglyc...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
1,loinc,tg,None,14447-7,Trigl; TG; Trigly; Triglycrides; Trig; Triglyc...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
2,loinc,tg,None,28554-4,Trigl; TG; Trigly; Triglycrides; Trig; Triglyc...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
3,loinc,tg,None,53527-8,Trigl in HDL2; TG; Trigly; Triglycrides; Trig;...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
4,loinc,tg,None,70260-5,Trigl Prt; PT-TRIG; TG; Trigly; Triglycrides; ...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
5,loinc,tg,None,70269-6,Trigl Plr fld; PF-TRIG; TG; Trigly; Triglycrid...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
6,loinc,tg,None,59571-0,Trigl; TG; Trigly; Triglycrides; Trig; Triglyc...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
7,loinc,tg,None,3047-8,Trigl+VLDL Ester; TG; Trigly; Triglycrides; Tr...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
8,loinc,tg,None,9619-8,Trigl; TG; Trigly; Triglycrides; Trig; Triglyc...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
9,loinc,tg,None,2096-6,Cholest; Chol; Choles; Lipid; Cholesterol tota...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC


## Reference table for hdl-related loinc codes

In [18]:
%%read_sql

create or replace temporary table st_loinc_ref2 as

select
    'loinc' as cat1,
    'tg' as cat2,
    'hdl' as cat3,
     loinc_num as value,
    RELATEDNAMES2 as description,
    current_date() as active_date,
    'swar' as active_reason,
    null as deactive_date,
    null as deactive_reason,
    'RWD_DB.RWD.PELICAN_E_LOINC' as source

    from 
       RWD_DB.RWD.PELICAN_E_LOINC
    
    where
        component ilike '%hdl%'

Query started at 07:51:08 PM India Standard Time; Query executed in 0.07 m

,status
0,Table ST_LOINC_REF2 successfully created.


In [19]:
%%read_sql df

select * from st_loinc_ref2

Query started at 07:51:17 PM India Standard Time; Query executed in 0.05 m

,cat1,cat2,cat3,value,description,active_date,active_reason,deactive_date,deactive_reason,source
0,loinc,tg,hdl,2086-7,HDLc; TCHHDL; HDL-C; High density lipoprotein ...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
1,loinc,tg,hdl,57937-5,HDLc 3a; TCHHDL; HDL-C; High density lipoprote...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
2,loinc,tg,hdl,18263-4,HDLc; TCHHDL; HDL-C; High density lipoprotein ...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
3,loinc,tg,hdl,53527-8,Trigl in HDL2; TG; Trigly; Triglycrides; Trig;...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
4,loinc,tg,hdl,16616-5,HDLc; TCHHDL; HDL-C; High density lipoprotein ...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
5,loinc,tg,hdl,35197-3,HDLc; TCHHDL; HDL-C; High density lipoprotein ...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
6,loinc,tg,hdl,27340-9,HDLc; TCHHDL; HDL-C; High density lipoprotein ...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
7,loinc,tg,hdl,57936-7,HDLc 2a; TCHHDL; HDL-C; High density lipoprote...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
8,loinc,tg,hdl,26015-8,HDL2c; HDLc; TCHHDL; HDL-C; High density lipop...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
9,loinc,tg,hdl,13460-1,LDLc; LDL-C; Low density lipoprotein cholester...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC


## Merging (union) tg and hdl reference tables

In [20]:
%%read_sql df

select * from st_loinc_ref1
union
select * from st_loinc_ref2

Query started at 07:51:28 PM India Standard Time; Query executed in 0.06 m

,cat1,cat2,cat3,value,description,active_date,active_reason,deactive_date,deactive_reason,source
0,loinc,tg,None,3047-8,Trigl+VLDL Ester; TG; Trigly; Triglycrides; Tr...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
1,loinc,tg,None,34695-7,VLDLc; Very low density lipoprotein cholestero...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
2,loinc,tg,None,35360-7,Lpa Trigl; Lpa Triglyceride; LP(a) Triglycerid...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
3,loinc,tg,None,3049-4,Trigl; TG; Trigly; Triglycrides; Trig; Triglyc...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
4,loinc,tg,None,3048-6,Trigl; TG; Trigly; Triglycrides; Trig; Triglyc...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
5,loinc,tg,None,1644-4,Trigl; TG; Trigly; Triglycrides; Trig; Triglyc...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
6,loinc,tg,None,47210-0,Trigl; TG; Trigly; Triglycrides; Trig; Triglyc...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
7,loinc,tg,None,12950-2,Trigl; TG; Trigly; Triglycrides; Trig; Triglyc...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
8,loinc,tg,None,14927-8,Trigl; TG; Trigly; Triglycrides; Trig; Triglyc...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC
9,loinc,tg,None,13899-0,Trigl; TG; Trigly; Triglycrides; Trig; Triglyc...,2018-12-17,swar,None,None,RWD_DB.RWD.PELICAN_E_LOINC


In [21]:
# Export to check, and then will inactivate codes that are not relevant and re-upload as final reference table
df.to_excel("out/loinccodes_check.xlsx", index=False)

## Upload into snowflake

In [24]:
loinc_ref = pd.read_excel("in/loinc_ref.xlsx")

In [25]:
snow.drop_table("st_loinc_ref")

DROP TABLE IF EXISTS sandbox_analytics.sandbox.st_loinc_ref;
Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Table sandbox.st_loinc_ref dropped!


In [26]:
snow.upload_dataframe(loinc_ref, "st_loinc_ref")

Upload successful!


In [27]:
%%read_sql

select *
from st_loinc_ref
where deactive_date is not null

Query started at 10:07:12 AM India Standard Time; Query executed in 0.06 m

,cat1,cat2,cat3,value,description,active_date,active_reason,deactive_date,deactive_reason,source
0,loinc,tg,None,3047-8,Trigl+VLDL Ester; TG; Trigly; Triglycrides; Tr...,2018-12-17,swar,2018-12-18,tg+vLDL,RWD_DB.RWD.PELICAN_E_LOINC
1,loinc,tg,None,34695-7,VLDLc; Very low density lipoprotein cholestero...,2018-12-17,swar,2018-12-18,vLDLc,RWD_DB.RWD.PELICAN_E_LOINC
2,loinc,tg,None,35360-7,Lpa Trigl; Lpa Triglyceride; LP(a) Triglycerid...,2018-12-17,swar,2018-12-17,Triglyceride in lipoprotein a [Mass/volume] in...,RWD_DB.RWD.PELICAN_E_LOINC
3,loinc,tg,None,12950-2,Trigl; TG; Trigly; Triglycrides; Trig; Triglyc...,2018-12-17,swar,2018-12-18,body fluids,RWD_DB.RWD.PELICAN_E_LOINC
4,loinc,tg,None,13899-0,Trigl; TG; Trigly; Triglycrides; Trig; Triglyc...,2018-12-17,swar,2018-12-18,Deprecated Triglyceride [Mass/volume] in fluid,RWD_DB.RWD.PELICAN_E_LOINC
5,loinc,tg,None,29766-3,Trigl; TG; Trigly; Triglycrides; Trig; Triglyc...,2018-12-17,swar,2018-12-18,Triglyceride [Moles/volume] in Body fluid,RWD_DB.RWD.PELICAN_E_LOINC
6,loinc,tg,None,59036-4,Trigl; TG; Trigly; Triglycrides; Trig; Triglyc...,2018-12-17,swar,2018-12-17,Triglyceride [Mass/volume] in Peritoneal dialy...,RWD_DB.RWD.PELICAN_E_LOINC
7,loinc,tg,None,53526-0,Trigl in VLDL; TG; Trigly; Triglycrides; Trig;...,2018-12-17,swar,2018-12-18,tg+vLDL,RWD_DB.RWD.PELICAN_E_LOINC
8,loinc,tg,None,3044-5,Trigl+HDL Ester; TG; Trigly; Triglycrides; Tri...,2018-12-17,swar,2018-12-18,Triglyceride+ester in HDL [Mass/volume] in Ser...,RWD_DB.RWD.PELICAN_E_LOINC
9,loinc,tg,None,51605-4,Trigl; TG; Trigly; Triglycrides; Trig; Triglyc...,2018-12-17,swar,2018-12-18,Triglyceride [Moles/volume] in Synovial fluid,RWD_DB.RWD.PELICAN_E_LOINC


# Patient cohorts
- Using laborders table

### Things to do:
- We want unique patients related to the above listed loinc codes.
- We then want to stratify them into buckets (see image in next section)
    - p.s.: these buckets have to be greater than or equal to

In [24]:
%%html
<img src="img/analysis_plan.png", width=800, height=800)>

## Observation cohort
- Including all patients who have a loinc code pertaining to tg or hdl

In [28]:
%%read_sql

create or replace table st_tg_obs_cohort as

select
    patient_id,
    loinc_num,
    result_status,
    report_date,
    obs_quan,
    obs_qual,
    unit

from
    RWD_DB.RWD.PELICAN_LABORDER
where
    loinc_num in (select value from st_loinc_ref where cat1 = 'loinc' and deactive_date is null)

Query started at 10:08:08 AM India Standard Time; Query executed in 0.65 m

,status
0,Table ST_TG_OBS_COHORT successfully created.


In [7]:
snow.select("select count(distinct patient_id) from st_tg_obs_cohort where left(patient_id, 5) != 'XXX -'")

,COUNT(DISTINCT PATIENT_ID)
0,7513315


In [29]:
%%read_sql
select * from st_tg_obs_cohort limit 100

Query started at 10:23:04 AM India Standard Time; Query executed in 0.10 m

,patient_id,loinc_num,result_status,report_date,obs_quan,obs_qual,unit
0,8BDCAEB0-2AB1-2F28-BBA9-D6C666A97600,2571-8,Final,2013-12-16,66.0,None,mg/dL
1,8BDCAEB0-2AB1-2F28-BBA9-D6C666A97600,2085-9,Final,2013-12-16,79.0,None,mg/dL
2,AF707EC3-1FA8-B7ED-8A46-D9FD8C388409,2085-9,Final,2018-11-01,47.0,None,mg/dL
3,AF707EC3-1FA8-B7ED-8A46-D9FD8C388409,2571-8,Final,2018-11-01,150.0,None,mg/dL
4,AF707EC3-1FA8-B7ED-8A46-D9FD8C388409,2085-9,Final,2018-07-01,41.0,None,mg/dL
5,AF707EC3-1FA8-B7ED-8A46-D9FD8C388409,2571-8,Final,2018-07-01,223.0,None,mg/dL
6,C354043B-E973-EBC0-28CD-1D64CE28F77B,2085-9,Final,2015-05-13,46.0,None,mg/dL
7,C99EC882-3033-9146-F916-46BB3058198B,2571-8,Final,2017-03-03,165.0,None,mg/dL
8,C99EC882-3033-9146-F916-46BB3058198B,2085-9,Final,2017-03-03,55.0,None,mg/dL
9,C99EC882-3033-9146-F916-46BB3058198B,2085-9,Final,2015-02-15,66.0,None,mg/dL


## Categorizing by triglyceride levels

- Every patient should have tg level and hdl level values
- Use latest report_date for distinct patient_id
- Ask if we want gender as a field?

In [14]:
snow.select("select distinct cat3 from st_loinc_ref")

,cat2
0,tg
1,hdl


### Triglyceride cohorts
- Selecting all patients with corresponding triglyceride level
- **Need 4 different tables for each category of triglycerides**

#### Tg levels lt 150 and gt=150

In [109]:
%%read_sql

create or replace table st_tg_cohort1 as

    select patient_id,
           report_date,
           obs_quan,
           unit,
           
           case
               when obs_quan < 150 then 'tg_lt150'
               when obs_quan >= 150 then 'tg_gt150'
               
           end as tg_level
       
    from st_tg_obs_cohort
    
        where loinc_num in (select value from st_loinc_ref where cat1 = 'loinc' and cat2 = 'tg' 
                            and cat3 is null and deactive_date is null)
               and result_status ilike '%Final%'  
               --and report_date = (select max(report_date) from st_tg_obs_cohort)

Query started at 04:25:06 PM India Standard Time; Query executed in 0.35 m

,status
0,Table ST_TG_COHORT1 successfully created.


In [101]:
snow.select("select * from st_tg_cohort1 limit 10")

,patient_id,report_date,obs_quan,unit,tg_level
0,C6A6FADB-55DE-3B13-1BAB-872046D74F9E,2018-11-04,134.0,mg/dL,tg_lt150
1,AFDD01E3-692F-CD3D-4766-891697A4DE81,2018-11-04,86.0,mg/dL,tg_lt150
2,CD813F7A-96B7-1909-11FC-15A5A5204064,2018-11-04,52.0,mg/dL,tg_lt150
3,8EA86E9A-C379-A096-EAB0-19B7D6CD26D0,2018-11-04,401.0,mg/dl,tg_gt150
4,823C97EA-BE80-ED29-09CE-C613E163BB0D,2018-11-04,146.6,mg/dL,tg_lt150
5,054105B9-ED84-E962-8293-B606AE6BA3C1,2018-11-04,144.0,mg/dL,tg_lt150
6,6CC71E57-06FB-4A8C-C1B3-F5E7380C5789,2018-11-04,113.0,mg/dl,tg_lt150
7,E855C4D3-4DFB-4BB3-DF14-4838C70FB87A,2018-11-04,61.0,MG/DL,tg_lt150
8,C5C69953-8236-4AAB-000C-8C7E85F0FB31,2018-11-04,110.0,mg/dL,tg_lt150
9,E03837F3-2C4F-17E5-BA08-DA5D0E71231F,2018-11-04,51.0,MG/DL,tg_lt150


In [111]:
%%read_sql

select  tg_level,
        count(distinct patient_id)
        
        from st_tg_cohort1
        
        --where unit ilike '%mg/dL%'
           --and hdl_unit ilike '%mg/dL%'
            
        group by tg_level
        --order by tg_level   

Query started at 04:33:06 PM India Standard Time; Query executed in 0.08 m

,tg_level,COUNT(DISTINCT PATIENT_ID)
0,None,46315
1,tg_lt150,5005397
2,tg_gt150,2372089


#### Tg levels >=180

In [112]:
%%read_sql

create or replace table st_tg_cohort2 as

    select patient_id,
           report_date,
           obs_quan,
           unit,
           
           case
               when obs_quan >= 180 then 'tg_gt180'
               
           end as tg_level
       
    from st_tg_obs_cohort
    
        where loinc_num in (select value from st_loinc_ref where cat1 = 'loinc' and cat2 = 'tg' 
                            and cat3 is null and deactive_date is null)
               and result_status ilike '%Final%'  
               --and report_date = (select max(report_date) from st_tg_obs_cohort)

Query started at 04:34:17 PM India Standard Time; Query executed in 0.24 m

,status
0,Table ST_TG_COHORT2 successfully created.


In [102]:
snow.select("select * from st_tg_cohort2 limit 10")

,patient_id,report_date,obs_quan,unit,tg_level
0,9A3AE741-223A-164A-BD57-1C3EA4DC2D59,2018-11-04,53.0,mg/dL,None
1,4A1F84EC-42C8-7253-0053-67140CB6C6F2,2018-11-04,78.0,mg/dL,None
2,23CF83B7-75D1-CAC5-77B4-6B9CFE9E74BA,2018-11-04,66.0,mg/dL,None
3,3B1E92D2-0E7B-A6A3-580E-F86419239DF0,2018-11-04,69.0,mg/dL,None
4,EF017586-A0D3-7D44-BE1F-7E9D530793E3,2018-11-04,214.0,mg/dL,tg_gt180
5,7B2CB51E-7763-1CD6-98D8-E99F91DEF2A5,2018-11-04,71.0,mg/dL,None
6,FA307D76-13C8-BDC4-A4D9-DF87CBADA914,2018-11-04,190.0,mg/dL,tg_gt180
7,5E874B1B-C785-89D1-699C-48BA221FBBEC,2018-11-04,289.0,mg/dL,tg_gt180
8,E2B013ED-0919-2480-A5E0-0840D4BE5F7C,2018-11-04,140.0,mg/dL,None
9,3DEE1949-31DC-42DF-E4C2-8E8D90E6A582,2018-11-04,146.0,mg/dL,None


In [113]:
%%read_sql

select  tg_level,
        count(distinct patient_id)
        
        from st_tg_cohort2
        
        --where unit ilike '%mg/dL%'
           --and hdl_unit ilike '%mg/dL%'
            
        group by tg_level
        --order by tg_level  

Query started at 04:36:56 PM India Standard Time; Query executed in 0.13 m

,tg_level,COUNT(DISTINCT PATIENT_ID)
0,None,5556695
1,tg_gt180,1678492


#### Tg levels >=200

In [114]:
%%read_sql

create or replace table st_tg_cohort3 as

    select patient_id,
           report_date,
           obs_quan,
           unit,
           
           case
               when obs_quan >= 200 then 'tg_gt200'
               
           end as tg_level
       
    from st_tg_obs_cohort
    
        where loinc_num in (select value from st_loinc_ref where cat1 = 'loinc' and cat2 = 'tg' 
                            and cat3 is null and deactive_date is null)
               and result_status ilike '%Final%'  
               --and report_date = (select max(report_date) from st_tg_obs_cohort)

Query started at 04:38:06 PM India Standard Time; Query executed in 0.14 m

,status
0,Table ST_TG_COHORT3 successfully created.


In [104]:
snow.select("select * from st_tg_cohort3 limit 20")

,patient_id,report_date,obs_quan,unit,tg_level
0,B21E265B-9584-FF3F-B689-D61CAB2BFE7A,2018-11-04,97.0,mg/dL,None
1,DF31E7D4-A12D-C21E-29F8-4664A2852C00,2018-11-04,167.0,mg/dL,None
2,615243F8-208A-2466-0D88-A040C0AE8D5D,2018-11-04,157.0,mg/dL,None
3,F00B8CB4-F30C-76D5-767A-A13ACAD6951D,2018-11-04,66.0,mg/dL,None
4,39C218C3-DB44-981C-A4BE-0478D5FE1A03,2018-11-04,36.0,mg/dL,None
5,C4E869E0-5E14-8635-C335-F554381FA55C,2018-11-04,353.0,mg/dL,tg_gt200
6,F32741AB-E168-4373-6CE4-1728CBECC87F,2018-11-04,51.0,mg/dL,None
7,742AD839-34C6-46F8-1A42-C12D2F2F820C,2018-11-04,49.0,mg/dL,None
8,A0E8CBBD-D809-53B8-6F4F-D17E4B6DD139,2018-11-04,167.0,mg/dL,None
9,45E060D4-6E80-CC39-6ED7-4AE5A74D9983,2018-11-04,103.0,mg/dL,None


In [115]:
%%read_sql

select  tg_level,
        count(distinct patient_id)
        
        from st_tg_cohort3
        
        --where unit ilike '%mg/dL%'
           --and hdl_unit ilike '%mg/dL%'
            
        group by tg_level
        --order by tg_level  

Query started at 04:38:24 PM India Standard Time; Query executed in 0.10 m

,tg_level,COUNT(DISTINCT PATIENT_ID)
0,tg_gt200,1337137
1,None,5782422


#### Tg levels >=500

In [116]:
%%read_sql

create or replace table st_tg_cohort4 as

    select patient_id,
           report_date,
           obs_quan,
           unit,
           
           case
               when obs_quan >= 500 then 'tg_gt500'
               
           end as tg_level
       
    from st_tg_obs_cohort
    
        where loinc_num in (select value from st_loinc_ref where cat1 = 'loinc' and cat2 = 'tg' 
                            and cat3 is null and deactive_date is null)
               and result_status ilike '%Final%'  
               --and report_date = (select max(report_date) from st_tg_obs_cohort)

Query started at 04:39:03 PM India Standard Time; Query executed in 0.11 m

,status
0,Table ST_TG_COHORT4 successfully created.


In [106]:
snow.select("select * from st_tg_cohort4 limit 50")

,patient_id,report_date,obs_quan,unit,tg_level
0,EC686422-E47F-DEA8-3232-34D8AD5149A5,2018-11-04,82.0,mg/dL,None
1,EFCDABEF-5B4F-CC75-AC10-110F1031F66B,2018-11-04,87.0,mg/dL,None
2,0B6F5417-EE1D-F8E1-EAA5-5DE7DA66F47D,2018-11-04,163.0,mg/dL,None
3,00900289-9F6F-274D-7B5F-29BE97284BF0,2018-11-04,83.0,mg/dL,None
4,3136FA35-6287-7C59-941A-4ECEC14B00BC,2018-11-04,159.0,mg/dL,None
5,D58CED07-2DEC-42AD-92E5-08C2691488FA,2018-11-04,159.0,None,None
6,49822F04-9471-FB96-A048-784C396D9719,2018-11-04,153.0,mg/dL,None
7,C50C5539-4D76-26CB-39FB-12990F8D3F0C,2018-11-04,69.0,mg/dL,None
8,39CC9D68-198F-1220-D7BE-06F366F3F1A3,2018-11-04,79.0,None,None
9,304B8F47-616F-7BFD-BA77-B269A0112AFB,2018-11-04,91.0,mg/dL,None


In [117]:
%%read_sql

select  tg_level,
        count(distinct patient_id)
        
        from st_tg_cohort4
        
        --where unit ilike '%mg/dL%'
           --and hdl_unit ilike '%mg/dL%'
            
        group by tg_level
        --order by tg_level  

Query started at 04:39:28 PM India Standard Time; Query executed in 0.05 m

,tg_level,COUNT(DISTINCT PATIENT_ID)
0,None,6413650
1,tg_gt500,109900


## HDL cohort
- creating table for HDL levels

In [118]:
%%read_sql

create or replace table st_hdl_cohort as

select 
    patient_id,
    report_date as hdl_report_date,
    obs_quan as hdl_value,
    unit,
    
    case 
        when obs_quan < 40 then 'hdl_lt40'
        when obs_quan >= 40 then 'hdl_gt40'
    
    end as hdl_level

from st_tg_obs_cohort

where loinc_num in (select value from st_loinc_ref where cat1 = 'loinc' and cat2 = 'tg' 
                            and cat3 = 'hdl' and deactive_date is null)
        and result_status ilike '%Final%'

Query started at 04:40:15 PM India Standard Time; Query executed in 0.13 m

,status
0,Table ST_HDL_COHORT successfully created.


In [70]:
snow.select("select * from st_hdl_cohort limit 50")

,patient_id,hdl_report_date,hdl_value,unit,hdl_level
0,96C7A2A2-4BCE-DCA0-CC95-E1ACBB40A1A9,2015-10-08,54.0,mg/dL,hdl_gt40
1,A7B38B8E-2D47-4712-3B1C-91DC9E13CC19,2017-12-13,60.0,mg/dL,hdl_gt40
2,A7B38B8E-2D47-4712-3B1C-91DC9E13CC19,2018-01-10,59.0,mg/dL,hdl_gt40
3,8779A09A-0C76-1010-D604-1A5B9F66BFA3,2017-04-22,43.0,mg/dL,hdl_gt40
4,4DB0B540-2043-F47C-DE5F-ADC208706BE8,2017-04-04,48.0,mg/dL,hdl_gt40
5,C2F17244-F79F-E892-0A2B-35204C9D70FA,2014-04-28,111.0,mg/dL,hdl_gt40
6,C2F17244-F79F-E892-0A2B-35204C9D70FA,2013-01-29,109.0,mg/dL,hdl_gt40
7,C2F17244-F79F-E892-0A2B-35204C9D70FA,2013-10-28,108.0,mg/dL,hdl_gt40
8,C2F17244-F79F-E892-0A2B-35204C9D70FA,2015-05-01,119.0,mg/dL,hdl_gt40
9,418D536D-DE81-6110-E482-4515325A840E,2018-03-13,43.0,mg/dL,hdl_gt40


## Final patient counts with tg levels and HDL
- All the records have been matched on the report_date. I thought of it as a comorbidity where at a given time point (in our case report_date) we want to know the patient's tg level and hdl level.

### Tg levels lt150 and gt150 and hdl counts

In [119]:
%%read_sql

create or replace table st_tg_hdl_agg1 as 
select a.patient_id,
        a.obs_quan,
     a.tg_level,
     a.unit,
     b.hdl_value,
     b.unit as hdl_unit,
     b.hdl_level
 from
    st_tg_cohort1 a
        join st_hdl_cohort b
            on a.patient_id = b.patient_id
            
            where a.report_date = b.hdl_report_date
--group by 1

Query started at 04:43:39 PM India Standard Time; Query executed in 0.17 m

,status
0,Table ST_TG_HDL_AGG1 successfully created.


In [120]:
snow.select("select * from st_tg_hdl_agg1 limit 5")

,patient_id,obs_quan,tg_level,unit,hdl_value,hdl_unit,hdl_level
0,D543699F-63F9-56BA-37FD-C38A1BA527A7,63.0,tg_lt150,mg/dL,40.0,mg/dL,hdl_gt40
1,D543699F-63F9-56BA-37FD-C38A1BA527A7,91.0,tg_lt150,mg/dL,42.0,mg/dL,hdl_gt40
2,DFA6DDAA-242E-4096-3748-A3F1C637D115,118.0,tg_lt150,mg/dL,46.0,mg/dL,hdl_gt40
3,094393F2-4A2B-393A-4951-D1C7837D81C1,93.0,tg_lt150,mg/dL,73.0,mg/dL,hdl_gt40
4,F748A4BF-2872-64A0-9E80-02122C444629,242.0,tg_gt150,mg/dL,45.0,mg/dL,hdl_gt40


In [122]:
%%read_sql

select  tg_level,
        hdl_level,
        count(distinct patient_id)
        
        from st_tg_hdl_agg1
        
        --where unit ilike '%mg/dL%'
           --and hdl_unit ilike '%mg/dL%'
            
        group by tg_level, hdl_level
        --order by tg_level  

Query started at 04:48:39 PM India Standard Time; Query executed in 0.08 m

,tg_level,hdl_level,COUNT(DISTINCT PATIENT_ID)
0,None,hdl_lt40,936
1,tg_gt150,hdl_gt40,1626026
2,tg_lt150,hdl_lt40,745365
3,None,hdl_gt40,3258
4,tg_gt150,hdl_lt40,977354
5,tg_gt150,None,2066
6,tg_lt150,hdl_gt40,4500249
7,tg_lt150,None,3055
8,None,None,43351


### Tg levels gt180 and hdl counts

In [123]:
%%read_sql

create or replace table st_tg_hdl_agg2 as 
select a.patient_id,
        a.obs_quan,
     a.tg_level,
     a.unit,
     b.hdl_value,
     b.unit as hdl_unit,
     b.hdl_level
 from
    st_tg_cohort2 a
        join st_hdl_cohort b
            on a.patient_id = b.patient_id
            
            where a.report_date = b.hdl_report_date
--group by 1

Query started at 04:50:56 PM India Standard Time; Query executed in 0.23 m

,status
0,Table ST_TG_HDL_AGG2 successfully created.


In [124]:
%%read_sql

select  tg_level,
        hdl_level,
        count(distinct patient_id)
        
        from st_tg_hdl_agg2
        
        --where unit ilike '%mg/dL%'
           --and hdl_unit ilike '%mg/dL%'
            
        group by tg_level, hdl_level
        --order by tg_level   

Query started at 05:08:30 PM India Standard Time; Query executed in 0.12 m

,tg_level,hdl_level,COUNT(DISTINCT PATIENT_ID)
0,None,hdl_lt40,953115
1,None,hdl_gt40,4896650
2,tg_gt180,None,1743
3,tg_gt180,hdl_gt40,1048525
4,tg_gt180,hdl_lt40,784308
5,None,None,45307


### Tg levels gt200 and hdl counts

In [125]:
%%read_sql

create or replace table st_tg_hdl_agg3 as 
select a.patient_id,
        a.obs_quan,
     a.tg_level,
     a.unit,
     b.hdl_value,
     b.unit as hdl_unit,
     b.hdl_level
 from
    st_tg_cohort3 a
        join st_hdl_cohort b
            on a.patient_id = b.patient_id
            
            where a.report_date = b.hdl_report_date
--group by 1

Query started at 05:11:11 PM India Standard Time; Query executed in 0.31 m

,status
0,Table ST_TG_HDL_AGG3 successfully created.


In [126]:
%%read_sql

select  tg_level,
        hdl_level,
        count(distinct patient_id)
        
        from st_tg_hdl_agg3
        
        --where unit ilike '%mg/dL%'
           --and hdl_unit ilike '%mg/dL%'
            
        group by tg_level, hdl_level
        --order by tg_level   

Query started at 05:12:10 PM India Standard Time; Query executed in 0.10 m

,tg_level,hdl_level,COUNT(DISTINCT PATIENT_ID)
0,None,hdl_lt40,1061297
1,tg_gt200,hdl_lt40,671657
2,None,hdl_gt40,5056237
3,tg_gt200,hdl_gt40,781917
4,tg_gt200,None,1576
5,None,None,45381


### Tg levels gt500 and hdl counts

In [127]:
%%read_sql

create or replace table st_tg_hdl_agg4 as 
select a.patient_id,
        a.obs_quan,
     a.tg_level,
     a.unit,
     b.hdl_value,
     b.unit as hdl_unit,
     b.hdl_level
 from
    st_tg_cohort4 a
        join st_hdl_cohort b
            on a.patient_id = b.patient_id
            
            where a.report_date = b.hdl_report_date
--group by 1

Query started at 05:13:25 PM India Standard Time; Query executed in 0.23 m

,status
0,Table ST_TG_HDL_AGG4 successfully created.


In [128]:
%%read_sql

select  tg_level,
        hdl_level,
        count(distinct patient_id)
        
        from st_tg_hdl_agg4
        
        --where unit ilike '%mg/dL%'
           --and hdl_unit ilike '%mg/dL%'
            
        group by tg_level, hdl_level
        --order by tg_level   

Query started at 05:13:43 PM India Standard Time; Query executed in 0.10 m

,tg_level,hdl_level,COUNT(DISTINCT PATIENT_ID)
0,None,hdl_lt40,1488844
1,tg_gt500,None,1002
2,tg_gt500,hdl_gt40,25086
3,None,hdl_gt40,5419485
4,tg_gt500,hdl_lt40,87867
5,None,None,45682


# Notes for Nish
- My main concern is that a patient diagnosed in say, 2015 with x level of tg and y level of hdl may not have the same values in 2017. Using a distinct function will pick the first record for that patient_id so it doesn't really resolve my problem.
- Right now, I am treating these buckets as exclusive of each other but I am not sure if it is the right approach. I tried using the max(report_date) function to get the latest EHR record but the results were very weird so I went back to a more simplistic kinda approach.